## 載入模型

In [386]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [387]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [388]:
# 載入模型與標準化器
base_model = load_model('DNN_best_model4mcAP.h5')

scaler = joblib.load('scaler_4mcAP.pkl')

In [389]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 64)                832       
                                                                 
 dense_45 (Dense)            (None, 128)               8320      
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 49)                3185      
                                                                 
Total params: 20,593
Trainable params: 3,185
Non-trainable params: 17,408
_________________________________________________________________


In [390]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [391]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
                              'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = ['AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
                              'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [392]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2024_12_21_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2024_12_21"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-11,444.0,6841.0,7353.0,5116.0,-60.0,-71.0,-50.0,-62.0,809.0,705.0,210.0,996.0
1,1-11,366.0,6841.0,7236.0,5242.0,-59.0,-71.0,-54.0,-66.0,808.0,766.0,719.0,420.0
2,1-11,288.0,8569.0,7217.0,4738.0,-58.0,-71.0,-56.0,-64.0,506.0,531.0,131.0,2157.0
3,1-11,366.0,6958.0,7327.0,6854.0,-59.0,-71.0,-52.0,-58.0,842.0,680.0,348.0,1181.0
4,1-11,327.0,6899.0,7275.0,6887.0,-59.0,-71.0,-54.0,-62.0,907.0,706.0,451.0,741.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18860,7-1,6284.0,2358.0,7783.0,594.0,-73.0,-74.0,-58.0,-58.0,849.0,991.0,933.0,576.0
18861,7-1,6372.0,2280.0,7627.0,516.0,-66.0,-75.0,-60.0,-59.0,1409.0,909.0,735.0,679.0
18862,7-1,6430.0,2202.0,7705.0,360.0,-73.0,-76.0,-60.0,-59.0,869.0,924.0,731.0,781.0
18863,7-1,6313.0,2280.0,7705.0,516.0,-73.0,-74.0,-60.0,-59.0,866.0,1198.0,750.0,598.0


In [393]:
# 資料前處理 (一): 刪除前後n筆資料
n = 20
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 1054
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4077.0,6303.0,13369.0,-460.0,-64.0,-74.0,-65.0,-53.0,1123.0,619.0,863.0,1536.0
1,1-1,1,4116.0,6421.0,13096.0,-391.0,-63.0,-75.0,-67.0,-52.0,1005.0,581.0,1066.0,1474.0
2,1-1,2,3843.0,6382.0,13164.0,-381.0,-64.0,-74.0,-69.0,-53.0,1260.0,785.0,961.0,1374.0
3,1-1,3,3882.0,6421.0,13223.0,-225.0,-64.0,-74.0,-69.0,-53.0,1631.0,691.0,953.0,1151.0
4,1-1,4,3960.0,6343.0,13135.0,-303.0,-63.0,-75.0,-67.0,-50.0,1081.0,577.0,1085.0,1362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16900,9-11,16900,1929.0,8569.0,2666.0,5086.0,-55.0,-68.0,-51.0,-70.0,216.0,77.0,400.0,50.0
16901,9-11,16901,1850.0,8569.0,2549.0,4191.0,-55.0,-68.0,-52.0,-68.0,289.0,360.0,431.0,832.0
16902,9-11,16902,1694.0,8511.0,2705.0,5057.0,-55.0,-68.0,-51.0,-70.0,358.0,133.0,364.0,48.0
16903,9-11,16903,1889.0,8374.0,2471.0,3690.0,-55.0,-68.0,-51.0,-65.0,272.0,165.0,541.0,1418.0


In [394]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [395]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])
# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

X_test

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,4077.0,6303.0,13369.0,-460.0,1123.0,619.0,863.0,1536.0,-64.0,-74.0,-65.0,-53.0
1,4116.0,6421.0,13096.0,-391.0,1005.0,581.0,1066.0,1474.0,-63.0,-75.0,-67.0,-52.0
2,3843.0,6382.0,13164.0,-381.0,1260.0,785.0,961.0,1374.0,-64.0,-74.0,-69.0,-53.0
3,3882.0,6421.0,13223.0,-225.0,1631.0,691.0,953.0,1151.0,-64.0,-74.0,-69.0,-53.0
4,3960.0,6343.0,13135.0,-303.0,1081.0,577.0,1085.0,1362.0,-63.0,-75.0,-67.0,-50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16900,1929.0,8569.0,2666.0,5086.0,216.0,77.0,400.0,50.0,-55.0,-68.0,-51.0,-70.0
16901,1850.0,8569.0,2549.0,4191.0,289.0,360.0,431.0,832.0,-55.0,-68.0,-52.0,-68.0
16902,1694.0,8511.0,2705.0,5057.0,358.0,133.0,364.0,48.0,-55.0,-68.0,-51.0,-70.0
16903,1889.0,8374.0,2471.0,3690.0,272.0,165.0,541.0,1418.0,-55.0,-68.0,-51.0,-65.0


In [396]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

print(len(X_test_scaled))

16905


### 重新訓練model

In [397]:
# 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
X_train_small, X_unused, y_train_small, y_unused = train_test_split(
    X_test_scaled, y_test_numeric, test_size=0.9, random_state=42, stratify=y_test_numeric
)

In [ ]:
import time
# 記錄開始時間
start_time = time.time()

In [ ]:
base_model.fit(X_train_small, y_train_small, epochs=20, batch_size=32)

Epoch 1/50


53/53 [==============================] - 0s 414us/step - loss: 4.8651 - accuracy: 0.4893
Epoch 2/50
53/53 [==============================] - 0s 477us/step - loss: 3.0819 - accuracy: 0.5556
Epoch 3/50
53/53 [==============================] - 0s 777us/step - loss: 2.0434 - accuracy: 0.6302
Epoch 4/50
53/53 [==============================] - 0s 617us/step - loss: 1.3868 - accuracy: 0.7183
Epoch 5/50
53/53 [==============================] - 0s 595us/step - loss: 0.9975 - accuracy: 0.7828
Epoch 6/50
53/53 [==============================] - 0s 602us/step - loss: 0.7582 - accuracy: 0.8361
Epoch 7/50
53/53 [==============================] - 0s 579us/step - loss: 0.6034 - accuracy: 0.8680
Epoch 8/50
53/53 [==============================] - 0s 518us/step - loss: 0.5040 - accuracy: 0.8899
Epoch 9/50
53/53 [==============================] - 0s 339us/step - loss: 0.4265 - accuracy: 0.9095
Epoch 10/50
53/53 [==============================] - 0s 327us/step - loss: 0.3679 - accuracy: 0.9243
Epoch 11/5

In [399]:
# 使用剩餘的 90% 測試資料進行模型評估
loss, accuracy = base_model.evaluate(X_unused, y_unused)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

476/476 [==============================] - 0s 323us/step - loss: 0.1818 - accuracy: 0.9730
Evaluation on 90% unused data - Loss: 0.1818, Accuracy: 0.9730


In [400]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [401]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

529/529 [==============================] - 0s 365us/step


['1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',


In [402]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4077.0,6303.0,13369.0,-460.0,-64.0,-74.0,-65.0,-53.0,1123.0,619.0,863.0,1536.0
1,1-1,1,4116.0,6421.0,13096.0,-391.0,-63.0,-75.0,-67.0,-52.0,1005.0,581.0,1066.0,1474.0
2,1-1,2,3843.0,6382.0,13164.0,-381.0,-64.0,-74.0,-69.0,-53.0,1260.0,785.0,961.0,1374.0
3,1-1,3,3882.0,6421.0,13223.0,-225.0,-64.0,-74.0,-69.0,-53.0,1631.0,691.0,953.0,1151.0
4,1-1,4,3960.0,6343.0,13135.0,-303.0,-63.0,-75.0,-67.0,-50.0,1081.0,577.0,1085.0,1362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16900,9-11,16900,1929.0,8569.0,2666.0,5086.0,-55.0,-68.0,-51.0,-70.0,216.0,77.0,400.0,50.0
16901,9-11,16901,1850.0,8569.0,2549.0,4191.0,-55.0,-68.0,-52.0,-68.0,289.0,360.0,431.0,832.0
16902,9-11,16902,1694.0,8511.0,2705.0,5057.0,-55.0,-68.0,-51.0,-70.0,358.0,133.0,364.0,48.0
16903,9-11,16903,1889.0,8374.0,2471.0,3690.0,-55.0,-68.0,-51.0,-65.0,272.0,165.0,541.0,1418.0


In [403]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2024_12_21.json

Test Data Mean MDE: 0.0365 meters
